In [17]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torchvision.models as models
from torchvision import transforms as T
import urllib


from PIL import Image
from PIL import ImageFilter 
import os 
from art.attacks.evasion import *
from art.estimators.classification import PyTorchClassifier

transform = T.Compose([            
 #T.Resize(256),                                    
 T.ToTensor(),                     
 T.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225])
 ])
model = models.resnet18(pretrained=True).eval()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=1000,
)

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [19]:
def loader(): 
 
    inPath ="/home/pawan/prism/cat/"
  

    outPath ="home/pawan/prism/output"
  
    for imagePath in os.listdir(inPath): 
 
        inputPath = os.path.join(inPath, imagePath) 

        img = Image.open(inputPath) 
        transformed_img = transform(img)
        input_img = transformed_img.unsqueeze(0)
        
        e = 0.5
        attack = DeepFool(classifier)
        advx = attack.generate(x=input_img)
        rescaled = (advx - advx.min()) / (advx.max() - advx.min())
        rescaled = (255*np.transpose(np.squeeze(rescaled))).astype(np.uint8)
        rescaled = np.moveaxis(rescaled ,0, 1)
        output = Image.fromarray(rescaled)
        
        fullOutPath = os.path.join(outPath, 'invert_'+imagePath) 
        output.save(fullOutPath)
        
        print(fullOutPath) 
  